In [1]:
!pip install sentence-transformers -q
!pip install --upgrade gensim -q
!pip install bertopic -q

In [2]:
import gensim
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import tqdm
import nltk
import re
from operator import itemgetter
import time
import multiprocessing
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

EPOCHS = 265
TOPICS = 4
CHUNK_SIZE = 1000
WORKERS = 7
EVAL_PERIOD = 10

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def preprocess_text(sentence):
    # Lowercase
    sentence = sentence.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub(r'[^a-zA-Z]+', ' ', sentence)
    sentence = sentence.replace('\n', '')
    # sentence = re.sub('\s\s+', ' ', sentence)
    
    # Tokenize & remove stop-words
    word_list = nltk.word_tokenize(sentence)    
    stopwords_list = set(nltk.corpus.stopwords.words('english'))
    word_list = [word for word in word_list if word not in stopwords_list]
    
    # Remove very small words, length < 3, they don't contribute any useful information
    word_list = [word for word in word_list if len(word) > 3]
        
    # Stem & Lemmatize
    porter_stemmer = nltk.stem.PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    word_list = [porter_stemmer.stem(word) for word in word_list]
    word_list = [lemmatizer.lemmatize(word) for word in word_list]
    
    sentence = ' '.join(word_list)
    
    return sentence

## Load Data

In [5]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [6]:
for index, row in train_df.iterrows():
    if row['class'] == 1:
        train_df.at[index, 'topic'] = 'World'
    elif row['class'] == 2:
        train_df.at[index, 'topic'] = 'Sports'
    elif row['class'] == 3:
        train_df.at[index, 'topic'] = 'Business'
    else:
        train_df.at[index, 'topic'] = 'Sci/Tech'

for index, row in test_df.iterrows():
    if row['class'] == 1:
        test_df.at[index, 'topic'] = 'World'
    elif row['class'] == 2:
        test_df.at[index, 'topic'] = 'Sports'
    elif row['class'] == 3:
        test_df.at[index, 'topic'] = 'Business'
    else:
        test_df.at[index, 'topic'] = 'Sci/Tech'

In [7]:
# Checking the average length of news article for each class
train_avg_len_by_class = (train_df.groupby('topic')['description'].apply(lambda x: np.mean(x.str.len())).reset_index(name='avg_news_len'))
test_avg_len_by_class = (test_df.groupby('topic')['description'].apply(lambda x: np.mean(x.str.len())).reset_index(name='avg_news_len'))

In [8]:
train_avg_len_by_class

,topic,avg_news_len
0,Business,198.690267
1,Sci/Tech,193.852533
2,Sports,185.171367
3,World,195.893900


In [9]:
test_avg_len_by_class

,topic,avg_news_len
0,Business,197.831053
1,Sci/Tech,193.951579
2,Sports,183.838947
3,World,194.010526


In [10]:
# Preprocess the news description
tqdm.tqdm.pandas()
train_df['news_tokenized'] = train_df['description'].progress_apply(lambda x: preprocess_text(str(x)))
test_df['news_tokenized'] = test_df['description'].progress_apply(lambda x: preprocess_text(str(x)))

100%|██████████| 7600/7600 [00:08<00:00, 865.04it/s]


### Sentence BERT

In [11]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
sbert_train_embeddings = embedder.encode(train_df['news_tokenized'])
sbert_test_embeddings = embedder.encode(test_df['news_tokenized'])

In [12]:
performance_metrics = pd.DataFrame(columns=['feature-extraction','clustering-algo', 'run#', 'state', 'AMI','ARI','NMI','time'])

### k-means

In [13]:
for run, state in zip(range(1, 21, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    k_means = KMeans(n_clusters=TOPICS, init='k-means++', max_iter=EPOCHS, random_state=state)
    k_means.fit(sbert_train_embeddings)
    
    pred_labels = k_means.fit_predict(sbert_test_embeddings)
        
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    stop = time.time()
    
    performance_metrics = performance_metrics.append({'feature-extraction':'Sentence BERT', 'clustering-algo':'k-means', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi, 'time':(stop-start)}, ignore_index=True)

Run # 1
Run # 2
Run # 3
Run # 4
Run # 5
Run # 6
Run # 7
Run # 8
Run # 9
Run # 10
Run # 11
Run # 12
Run # 13
Run # 14
Run # 15
Run # 16
Run # 17
Run # 18
Run # 19
Run # 20


In [14]:
performance_metrics

,feature-extraction,clustering-algo,run#,state,AMI,ARI,NMI,time
0,Sentence BERT,k-means,1,2,0.498843,0.520087,0.499059,37.054173
1,Sentence BERT,k-means,2,4,0.499652,0.518302,0.499867,47.199358
2,Sentence BERT,k-means,3,6,0.499513,0.519848,0.499728,40.775629
3,Sentence BERT,k-means,4,8,0.499652,0.518302,0.499867,36.421963
4,Sentence BERT,k-means,5,10,0.498689,0.517889,0.498904,36.470073
5,Sentence BERT,k-means,6,12,0.499340,0.521579,0.499555,48.355836
6,Sentence BERT,k-means,7,14,0.497790,0.517151,0.498006,58.114293
7,Sentence BERT,k-means,8,16,0.499607,0.518352,0.499822,36.345726
8,Sentence BERT,k-means,9,18,0.499549,0.517998,0.499765,36.389335
9,Sentence BERT,k-means,10,20,0.498844,0.520394,0.499059,45.455332


### BERTopic

In [15]:
for run, state in zip(range(1, 21, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2", verbose=True, nr_topics=4).fit(train_df['news_tokenized'], y=train_df['class'])
    
    pred_labels = topic_model.transform(test_df['news_tokenized'])
        
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels[0])
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels[0])
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels[0])
    stop = time.time()
    
    performance_metrics = performance_metrics.append({'feature-extraction':'Sentence BERT', 'clustering-algo':'BERTopic', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi, 'time':(stop-start)}, ignore_index=True)
    # break

Run # 1


Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

2021-12-13 17:19:08,702 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2021-12-13 17:23:15,940 - BERTopic - Reduced dimensionality with UMAP
2021-12-13 17:23:30,734 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-12-13 17:24:45,673 - BERTopic - Reduced number of topics from 1333 to 5


Batches:   0%|          | 0/238 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


...

Run # 20


Batches:   0%|          | 0/3750 [00:00<?, ?it/s]

2021-12-13 19:56:40,261 - BERTopic - Transformed documents to Embeddings
2021-12-13 20:00:34,007 - BERTopic - Reduced dimensionality with UMAP
2021-12-13 20:00:47,877 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-12-13 20:02:02,595 - BERTopic - Reduced number of topics from 1325 to 5


Batches:   0%|          | 0/238 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [17]:
performance_metrics.to_csv('evaluating_performance-sbert.csv', index=False)